In [1]:
%pylab notebook

import librosa
import IPython.display as display

import sklearn.decomposition as decomp
import sklearn.cluster as cluster

import tensorflow as tf
from tensorflow.keras import layers

Populating the interactive namespace from numpy and matplotlib


In [2]:
# lifted from https://keras.io/examples/lstm_text_generation/
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [3]:
# load and normalize data
frames, rate = librosa.load('./thuille.wav', sr=5000, mono=True)
frames -= frames.min()
frames /= frames.max()

In [4]:
display.Audio(frames, rate=rate)

In [5]:
# extract overlapping patches from audio
patch_len = 200
patch_skip = 100
patch_starts = range(0, len(frames)-patch_len, patch_skip)

patches = np.array([
    frames[i:i+patch_len]
    for i in patch_starts
])

In [6]:
# transform patches to ICA basis
n_components = 128
ica = decomp.FastICA(n_components)
patches_ = ica.fit_transform(patches)

In [7]:
# quantize patches to cluster IDs
n_clusters = 2048
kmeans = cluster.KMeans(n_clusters)
labels = kmeans.fit_predict(patches_)

In [8]:
# test reconstruction from quantized patches -> audio
recon = np.matmul(kmeans.cluster_centers_[labels], ica.mixing_.T) + ica.mean_
display.Audio(recon[:,:patch_skip].flatten(), rate=rate)

In [9]:
# extract sequences of cluster IDs for training
seq_len = 50
seq_skip = 1
seq_starts = range(0, len(labels)-seq_len, seq_skip)

sequences = np.array([
    labels[i:i+seq_len] for i in seq_starts
])

# for each sequence, one-hot encode cluster ID of start of next sequence
next_labels = np.zeros(shape=[len(sequences), n_clusters])
for i in seq_starts:
    next_labels[i, labels[i+seq_len]] = 1

In [10]:
model = tf.keras.Sequential([
    # embed cluster ID in 128-dim vector space
    layers.Embedding(input_dim=n_clusters, output_dim=128, input_length=seq_len),
    
    # 256-unit LSTM
    layers.LSTM(256),
    
    # predict next cluster ID
    layers.Dense(n_clusters, activation='softmax')
])

In [11]:
# train model
model.compile(tf.keras.optimizers.RMSprop(1e-2), 'categorical_crossentropy')
history = model.fit(sequences, next_labels, batch_size=128, epochs = 50)

/home/kstwrt/tensorflow-gpu/local/lib/python2.7/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/50
3247/3247 [==============================] - 2s 654us/step - loss: 8.6533
Epoch 2/50
3247/3247 [==============================] - 1s 384us/step - loss: 7.8255
Epoch 3/50
3247/3247 [==============================] - 1s 388us/step - loss: 7.4692
Epoch 4/50
3247/3247 [==============================] - 1s 381us/step - loss: 7.0293
Epoch 5/50
3247/3247 [==============================] - 1s 387us/step - loss: 6.1376
Epoch 6/50
3247/3247 [==============================] - 1s 390us/step - loss: 5.7833
Epoch 7/50
3247/3247 [==============================] - 1s 386us/step - loss: 5.3389
Epoch 8/50
3247/3247 [==============================] - 1s 383us/step - loss: 4.7265
Epoch 9/50
3247/3247 [==============================] - 1s 390us/step - loss: 3.8948
Epoch 10/50
3247/3247 [==============================] - 1s 385us/step - loss: 2.9502
Epoch 11/50
3247/3247 [==============================] - 1s 383us/step - loss: 2.0284
Epoch 12/50
3247/3247 [==============================] - 1s 386

In [12]:
# choose random sequence as seed
generated = sequences[[np.random.choice(len(sequences))]]

# add 1000 cluster IDs
for i in range(1000):
    # get last seq_len IDs
    inp = generated[:, -seq_len:]
    
    # predict next ID (with some noise)
    out = sample(model.predict(inp)[0], 1.0).reshape(1,1)
    
    # add to end of generated sequence
    generated = np.concatenate([generated, out], axis=1)

In [13]:
# reconstruct signal
recon = np.matmul(kmeans.cluster_centers_[generated[0]], ica.mixing_.T) + ica.mean_

In [14]:
# play from end of seed
display.Audio(
    recon[seq_len:, :patch_skip].flatten(),
    rate = rate
)